In [1]:
import json

def get_tickets():
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data

def get_ticket(index):
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data['tickets'][index]

def stringify_ticket(ticket):
    return f"Title: {ticket['title']}\nDescription: {ticket['description']}"

ticket = get_ticket(1)
print(stringify_ticket(ticket))


Title: Request to Block Cases from Specific Region
Description: I need to stop receiving cases from the APAC region as I don't have the language capabilities to handle them effectively. Is there a way to update my assignment rules to exclude these cases?


In [59]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
import markdown


def load_knowledge_base():
    with open('knowledge_wiki.md', 'r') as file:
        kb_content = file.read()
    
    chunks = [chunk.strip() for chunk in kb_content.split('###') if chunk.strip()]
    
    clean_chunks = []
    for chunk in chunks:
        html = markdown.markdown(chunk)
        clean_text = html.replace('<strong>', '').replace('</strong>', '') \
                        .replace('<p>', '').replace('</p>', '') \
                        .replace('<h3>', '').replace('</h3>', '') \
                        .replace('<ol>', '').replace('</ol>', '') \
                        .replace('<li>', '').replace('</li>', '')
        clean_chunks.append(clean_text)
    
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(
        clean_chunks,
        embeddings,
        metadatas=[{"chunk_id": i, "source": "knowledge_wiki"} for i in range(len(clean_chunks))]
    )
    return vectorstore


def find_relevant_knowledge(ticket_string, vectorstore, top_k=3):
    relevant_chunks = vectorstore.similarity_search(ticket_string, k=top_k)
    return [doc.page_content for doc in relevant_chunks]

In [60]:
vectorstore = load_knowledge_base()

In [62]:
ticket = get_ticket(2)
ticket_string = stringify_ticket(ticket)
print("Ticket: ", ticket_string)
relevant_knowledge = find_relevant_knowledge(ticket_string, vectorstore)
print("\nRelevant knowledge base sections:")
for i, chunk in enumerate(relevant_knowledge, 1):
    print(f"\nChunk {i}:")
    print(chunk)

Ticket:  Title: Time Zone Update Required
Description: I recently relocated to the EST time zone, but my working hours are still showing in PST. Need assistance updating my profile to reflect the correct time zone so my availability is properly tracked.

Relevant knowledge base sections:

Chunk 1:
Updating Time Zone Settings
To update your working hours and reflect the correct time zone:
1. Go to Profile Settings: Locate the "Time Zone" section.
2. Select the Correct Time Zone: Choose your current time zone from the dropdown.
3. Verify Your Schedule: Ensure that your work hours align with the new time zone.
4. Save Changes: Refresh the system and test by checking availability.

Chunk 2:
Troubleshooting Auto Assignment Issues
If you are not receiving auto assignments despite being marked as "Available," follow these steps:
1. Verify System Status: Check for ongoing outages or maintenance in the assignment system.
2. Check Queue Configuration: Ensure your queue is properly configured and

In [65]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

chat_model = ChatOllama(model="llama3.2:1b")

def generate_response(ticket_string, relevant_knowledge):
    messages = [
        SystemMessage(content="You are a helpful AI assistant. Use the relevant knowledge to solve the ticket."),
        HumanMessage(content="Ticket: " + ticket_string),
        HumanMessage(content="Relevant knowledge: " + relevant_knowledge)
    ]
    response = chat_model.invoke(messages)
    return response.content

response = generate_response(ticket_string, relevant_knowledge[0])
print(response)

I'd be happy to help you update your working hours and reflect the correct time zone in EST (Eastern Standard Time).

To start, I'll guide you through the steps:

1. Go to Profile Settings
2. Locate the "Time Zone" section
3. Select the Correct Time Zone: Choose your current time zone from the dropdown

Let's get started!


In [66]:
def solve_ticket(ticket_index):
    vectorstore = load_knowledge_base()
    
    ticket = get_ticket(ticket_index)
    ticket_string = stringify_ticket(ticket)

    relevant_knowledge = find_relevant_knowledge(ticket_string, vectorstore)
    response = generate_response(ticket_string, relevant_knowledge[0])

    return response

print(solve_ticket(0))

Based on the ticket description, I'll provide step-by-step troubleshooting to help resolve the auto assignment issue.

**Step 1: Verify System Status**
Check the system status to ensure there are no ongoing outages or maintenance in the assignment system. You can try checking the system's status page or contacting the system administrator to confirm that everything is operating smoothly.

**Step 2: Check Queue Configuration**
Verify that your queue is properly configured and there are no conflicting filters. Make sure that:

* Your queue is enabled and not set to "Hidden"
* There are no duplicate or conflicting queues
* The filter criteria (e.g., job type, location) are correct

**Step 3: Manually Refresh Status**
 Toggle your availability off and on again to refresh your status in the system. This will help you check if the issue is related to a temporary system glitch.

**Additional Steps**

* If none of the above steps resolve the issue, it's possible that there's a technical issue 